## Exercise 64 - Min/Max + Filter

* Textual file (past years) : Timestamp,StockId,Price
* Stream of readings : Timestamp,StockId,Price

Every 1 minute, by considering only the data received in the last 1 minute, print on the standard output the StockIDs of the stocks that satisfy one of the following conditions: 
* price of the stock < minimum historical price for that stock
* price of the stock > maximum historical price for that stock

In [2]:
from pyspark.streaming import StreamingContext

In [ ]:
# read the static file containing the stations
inputStockPrices = "/data/students/bigdata-01QYD/ex_data/Ex64/data/"
historicalStockPricesRDD = sc.textFile(inputStockPrices)

# extract (StockId,(Price,Price)
pairsHistoricalStockPricesRDD = historicalStockPricesRDD.map(lambda line : (  line.split(",")[1], 
                                                                              (float(line.split(",")[2]),float(line.split(",")[2]))  )
                                                            ).cache()
# compute the minimum and the maximum price for each stock
minMaxStockPrices = pairsHistoricalStockPricesRDD.reduceByKey(lambda p1,p2 : (min(p1[0],p2[0]), max(p1[1],p2[1])))

In [ ]:
# create the spark object that wait for the 
# information every two seconds
ssc = StreamingContext(sc,60)

# the stream will be connected to localhost
# through port 9999
lineDStream = ssc.SocketTextStream("localhost",9999)

In [ ]:
# emit (StockId, (price,price))
stocksDStream = lineDStream.map(lambda line: (line.split(",")[0], (float(line.split(",")[-1]),float(line.split(",")[-1])))

# compute the local min and max price for each stock because:
# - we reduce the number of values to compare
# - we don't have duplicates
# - we have much less joins
minMaxStockPricesDStream = stocksDStream.reduceByKey(lambda p1,p2 : (min(p1[0],p2[0]), max(p1[1],p2[1])))

# we need to use transform because we cannot compute the join between an RDD and a DStream
# it will return (StockID,(price,(minPrice,maxPrice)))
joinedStocksDStream = stocksDStream.transform(lambda batchRDD : batchRDD.join(minMaxStockPrices))

# filter the prices according to the request
# check if the "DStream" min price is < historical min price and vice versa
filteredStocksDStream = joinedNamesTimestampDStream.filter(lambda pair : pair[1][0][0] < pair[1][1][0] or 
                                                                         pair[1][0][0] > pair[1][1][1])

# since this condition might be satisfied more than once, we need to use also the distinct()
uniqueFilteredStocksDStream = filteredStocksDStream.transform(lambda batchRDD : batchRDD.keys().distinct())
                                               
# print on the standard output
filteredStocksDStream.pprint()

In [ ]:
# start reading the incoming data
ssc.start()

In [ ]:
# stay alive for at most 90 seconds
ssc.awaitTerminationOrTimeout(90)
# stop only the StreamingContext
# but not the SparkContext
ssc.stop(stopSparkContext=False)